### Data Analys

In [151]:
import pandas as pd
import numpy as np
import requests

In [ ]:
Visits = requests.get("https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01")
regs = requests.get("https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01")
Visits = pd.DataFrame(Visits.json())
regs = pd.DataFrame(regs.json())

##### Clean and sort Visits

In [164]:
visits_cleaned = Visits[Visits["platform"] != "bot"]
visits_sorted = visits_cleaned.sort_values(by="datetime")
visits_sorted = visits_cleaned.drop_duplicates(subset=["visit_id"], keep="last")

##### Unite dataframes

In [176]:
United = pd.merge(visits_sorted, regs, on=["datetime", "platform"], how="outer")
United["datetime"] = United["datetime"].str.slice(0,10)

##### Make a conversion dataframe

In [182]:
conversion = United.groupby(["datetime", "platform"], as_index=False).agg(visits = ("visit_id", "count"), registrations = ("user_id", "count"))
conversion = conversion.rename(columns = {"datetime": "date_group"})
conversion["conversion"] = conversion["registrations"] / conversion["visits"] * 100

##### Save conversion

In [183]:
conversion.to_json("./conversion.json")